In [1]:
import sqlite3
from pyteomics import mass as massC
import struct
import numpy as np
import pandas as pd

import keras.backend as K
from keras.layers.convolutional import Conv1D
from keras.layers.core import Dense, Dropout, Masking
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.models import load_model as keras_load_model
from keras.models import Sequential


Using TensorFlow backend.
/home/bsuwirjo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bsuwirjo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bsuwirjo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bsuwirjo/.local/lib/python3.6/site-packages/tensorflo

In [2]:
def binarySearch (arr, l, r, x, tol):

    # Check base case
    if r >= l:
        mid = int(l + (r - l)/2)

        # If element is present at the middle itself
        if arr[mid][0] > x - tol and arr[mid][0] < x + tol:
            return mid

        # If element is smaller than mid, then it can only
        # be present in left subarray
        elif arr[mid][0] > x + tol:
            return binarySearch(arr, l, mid-1, x, tol)

        # Else the element can only be present in right subarray
        else:
            return binarySearch(arr, mid+1, r, x, tol)

    else:
        # Element is not present in the array
        return -1

In [3]:
#Healper Function, Converts bitstring to float
def convertFloat(element):
    return struct.iter_unpack('>f', element)

In [4]:
def lossConvert(loss, charge):
     if loss == '':
         return 0
     elif loss == 'n':
         return massC.calculate_mass(formula='NH3', charge = charge)
     elif loss == 'o':
         return massC.calculate_mass(formula='H2O', charge = charge)

def getIonMasses(peptide, types=('b', 'y'), maxcharge=2):
    """
    The function generates all possible m/z for fragments of types
    `types` and of charges from 1 to `maxharge`.
    """
    ions = {"b1": [], "b2": [], "bn1": [], "bn2": [], "bo1": [], "bo2": [], "y1": [], "y2": [], "yn1": [], "yn2": [], "yo1": [], "yo2": [], }
    losses = ['', 'n', 'o']
    for ion_type in types:
        for charge in range(1, maxcharge+1):
            for lossT in losses:
                key = ion_type  + lossT + str(charge)
                loss = lossConvert(lossT, charge)
                for i in range(1, len(peptide)-1):
                    if ion_type[0] in 'abc':
                        ions[key].append( massC.fast_mass(peptide[:i], ion_type=ion_type, charge=charge))
                    else:
                        ions[key].append( massC.fast_mass(peptide[i:], ion_type=ion_type, charge=charge))
    return ions


In [5]:
#Connect to database
#conn = sqlite3.connect('/data/tyrande/data/MudPit/projects2012_01_05_20_21166.db')
conn = sqlite3.connect('testLibDuplicateSpectra.db')
xtrainc2 = []
xtrainc3 = []
ytrainc2 = []
ytrainc3 = []

c2Arr = []
c3Arr = []

#peptideTemp = {"peptide":"", "modification": 'null', "ions":{}}
peptideTemp = {"peptide":"", "ions":{}}

peptideCnt = 0
specCnt = 0

print ("Pulling Data from Database")
c = conn.cursor()
c.execute("SELECT * FROM PeptideTable")
pepTable = c.fetchall()

cnt = 0

#Iterate through table and pull information about each peptide and its scans
#for ind in pepTable:
for j in range(15):

    #Match peptide in table to peptide in Spectra Table
    #pepID = (str(ind[0]), )
    pepID = (str(pepTable[j][0]), )

    #peptide = ind[1].split('.')[1]
    peptide = pepTable[j][1].split('.')[1]

    if '(' in peptide or 'Z' in peptide or 'B' in peptide or 'U' in peptide:
        continue

    #charge = int(ind[4])
    charge = pepTable[j][4]

    #print(pepID, ":" ,peptide)

    peptideCnt += 1;
    c.execute('SELECT *,rowid FROM SpectraTable WHERE peptideID=?', pepID)
    spectrums = c.fetchall()

    #Make sure peptide has more than 50 spectrums for accurate training.
    specCnt += 1 #increment peptide count

    #Search returned list of matched spectrums for each peptide
    for element in spectrums:
        ions = getIonMasses(peptide)

        spectrum = [] #Holder variable for spectrum

        #append the scan id
        #idList.append(element[4])
        #massList.append(float(element[3])/1000);
        #Grab mzArr and intArr from
        mzArr = list(convertFloat(element[1]))
        intArr = list(convertFloat(element[2]))

        for key, massList in ions.items():
            intensities = []
            for mass in massList:
                #print(key,":" ,mass)
                found = False
                mass = mass
                tolerance = 400 * mass/1000000
                #cdistance = 10000
                #cValue = 0

                result = binarySearch(mzArr, 0, len(mzArr) - 1, mass, tolerance)
                if result != -1:
                    intensities.append(intArr[result][0])
                else:
                    intensities.append(0)
            ions[key] = intensities


        tmp = peptideTemp
        tmp['peptide'] = peptide
        tmp['ions'] = ions
        #print(tmp)


        if charge == 2:
            #xtrainc2.append(peptide)
            #ytrainc2.append(ions)
            c2Arr.append(tmp)
        elif charge == 3:
            #xtrainc3.append(peptide)
            #ytrainc3.append(ions)
            c3Arr.append(tmp)
        else:
            cnt += 1
            #print("Charge:", charge, "invalid")
print("Done")

Pulling Data from Database
Done


In [6]:
df = pd.DataFrame(c2Arr)
df = df.sample(frac=1)
df.head()
print(df.shape)

"""
for i in df["ions"]:
    for key, value in i.items():
        print(value)
"""

(740, 2)


'\nfor i in df["ions"]:\n    for key, value in i.items():\n        print(value)\n'

In [7]:

#Encode Sequence

#read the matrix a csv file on github
nlf = pd.read_csv('https://raw.githubusercontent.com/dmnfarrell/epitopepredict/master/epitopepredict/mhcdata/NLF.csv',index_col=0)

def show_matrix(m):
    #display a matrix
    cm = sns.light_palette("seagreen", as_cmap=True)
    display(m.style.background_gradient(cmap=cm))

def nlf_encode(seq):    
    x = pd.DataFrame([nlf[i] for i in seq]).reset_index(drop=True)  
    #show_matrix(x)
    e = x.values.flatten()
    return e

modPeptides = df['peptide'].copy()
for i, pep in enumerate(modPeptides):
    modPeptides.iloc[i] = nlf_encode(pep)

"""
for i in modPeptides:
    print(i.shape)
"""

#m odPeptides.head()
#print(modPeptides.iloc[0])

385    [0.7, 0.95, 0.36, 0.6, 0.09, 0.06, 0.55, 0.01,...
243    [0.7, 0.95, 0.36, 0.6, 0.09, 0.06, 0.55, 0.01,...
731    [0.7, 0.95, 0.36, 0.6, 0.09, 0.06, 0.55, 0.01,...
557    [0.7, 0.95, 0.36, 0.6, 0.09, 0.06, 0.55, 0.01,...
156    [0.7, 0.95, 0.36, 0.6, 0.09, 0.06, 0.55, 0.01,...
Name: peptide, dtype: object

import seaborn as sns
codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
#Encode Sequence
def show_matrix(m):
    #display a matrix
    cm = sns.light_palette("seagreen", as_cmap=True)
    display(m.style.background_gradient(cmap=cm))

def one_hot_encode(seq):
    o = list(set(codes) - set(seq))
    s = pd.DataFrame(list(seq))    
    x = pd.DataFrame(np.zeros((len(seq),len(o)),dtype=int),columns=o)    
    a = s[0].str.get_dummies(sep=',')
    a = a.join(x)
    a = a.sort_index(axis=1)
    #show_matrix(a)
    e = a.values.flatten()
    return e

modPeptides = df['peptide'].copy()
for i, pep in enumerate(modPeptides):
    modPeptides.iloc[i] = one_hot_encode(pep)

"""
for i in modPeptides:
    print(i.shape)
"""

modPeptides.head()
print(modPeptides.iloc[0])

In [8]:
df = pd.concat([df, modPeptides], axis = 1)
df.columns = ['sequence', 'ions', 'encodedSequence']
df = df[['sequence', 'encodedSequence', 'ions']]
df.head()
print(np.array(df['encodedSequence'].iloc[0]).shape)

(396,)


In [9]:
ionsProcessed = df['ions'].copy()
for i, ionDict in enumerate(ionsProcessed):
    container  = []
    for key,value in ionDict.items():
        ionList = []
        for j in value:
            ionList.append(j)
        container.append(np.array(ionList))
    ionsProcessed.iloc[i] = np.array(container)
    #print(np.array(container).shape)
#ionsProcessed.head()

df = pd.concat([df, ionsProcessed], axis = 1)
df.columns = ['sequence', 'encodedSequence', 'ions', 'ionsProcessed']
df.head()

,sequence,encodedSequence,ions,ionsProcessed
385,YIREPEHPASFYEVLYFQDPQA,"[0.7, 0.95, 0.36, 0.6, 0.09, 0.06, 0.55, 0.01,...","{'b1': [0, 0, 82.9000015258789, 70.90000152587...","[[0.0, 0.0, 82.9000015258789, 70.9000015258789..."
243,YIREPEHPASFYEVLYFQDPQA,"[0.7, 0.95, 0.36, 0.6, 0.09, 0.06, 0.55, 0.01,...","{'b1': [0, 0, 82.9000015258789, 70.90000152587...","[[0.0, 0.0, 82.9000015258789, 70.9000015258789..."
731,YIREPEHPASFYEVLYFQDPQA,"[0.7, 0.95, 0.36, 0.6, 0.09, 0.06, 0.55, 0.01,...","{'b1': [0, 0, 82.9000015258789, 70.90000152587...","[[0.0, 0.0, 82.9000015258789, 70.9000015258789..."
557,YIREPEHPASFYEVLYFQDPQA,"[0.7, 0.95, 0.36, 0.6, 0.09, 0.06, 0.55, 0.01,...","{'b1': [0, 0, 82.9000015258789, 70.90000152587...","[[0.0, 0.0, 82.9000015258789, 70.9000015258789..."
156,YIREPEHPASFYEVLYFQDPQA,"[0.7, 0.95, 0.36, 0.6, 0.09, 0.06, 0.55, 0.01,...","{'b1': [0, 0, 82.9000015258789, 70.90000152587...","[[0.0, 0.0, 82.9000015258789, 70.9000015258789..."


In [10]:

inputArr = np.array(df['encodedSequence'])
inputArr = np.array([np.array(i) for i in inputArr])
outputArr = np.array([np.array(i) for i in df['ionsProcessed']])

print(outputArr.shape)
inputArr = np.reshape(inputArr, (inputArr.shape[0], 1, inputArr.shape[1]))
outputArr = np.reshape(outputArr, (outputArr.shape[0], 240))

splitIdx = int(len(inputArr) * .1)

xTrain = inputArr[:splitIdx]
xTest = inputArr[splitIdx:]

yTrain = outputArr[:splitIdx]
yTest = outputArr[splitIdx:]
print(inputArr.shape)
print(outputArr.shape)


(740, 12, 20)
(740, 1, 396)
(740, 240)


In [11]:
model = Sequential()

model.add(Bidirectional(LSTM(3, input_shape = xTrain.shape)))
#model.add(Dense(20))
#model.add(Dropout(0.5))
#model.add(TimeDistributed(Dense(240, activation='relu')))
model.add(Dense(240, activation='relu'))

model.compile(
    loss="mean_squared_error",
    optimizer="adam")
model.fit(xTrain, yTrain, epochs = 1000)


Epoch 1/1000
74/74 [==============================] - 1s 8ms/step - loss: 1965406.1622
Epoch 2/1000
74/74 [==============================] - 0s 220us/step - loss: 1965391.1689
Epoch 3/1000
74/74 [==============================] - 0s 235us/step - loss: 1965374.7601
Epoch 4/1000
74/74 [==============================] - 0s 228us/step - loss: 1965358.0304
Epoch 5/1000
74/74 [==============================] - 0s 261us/step - loss: 1965341.4020
Epoch 6/1000
74/74 [==============================] - 0s 246us/step - loss: 1965325.7365
Epoch 7/1000
74/74 [==============================] - 0s 237us/step - loss: 1965310.7534
Epoch 8/1000
74/74 [==============================] - 0s 255us/step - loss: 1965296.7500
Epoch 9/1000
74/74 [==============================] - 0s 241us/step - loss: 1965283.5473
Epoch 10/1000
74/74 [==============================] - 0s 175us/step - loss: 1965271.0304
Epoch 11/1000
74/74 [==============================] - 0s 183us/step - loss: 1965259.0811
Epoch 12/1000
74/74 

74/74 [==============================] - 0s 220us/step - loss: 1964366.6149
Epoch 90/1000
74/74 [==============================] - 0s 269us/step - loss: 1964355.9493
Epoch 91/1000
74/74 [==============================] - 0s 169us/step - loss: 1964345.2736
Epoch 92/1000
74/74 [==============================] - 0s 195us/step - loss: 1964334.4527
Epoch 93/1000
74/74 [==============================] - 0s 208us/step - loss: 1964323.8615
Epoch 94/1000
74/74 [==============================] - 0s 223us/step - loss: 1964313.1318
Epoch 95/1000
74/74 [==============================] - 0s 216us/step - loss: 1964302.4527
Epoch 96/1000
74/74 [==============================] - 0s 239us/step - loss: 1964291.7905
Epoch 97/1000
74/74 [==============================] - 0s 146us/step - loss: 1964281.1318
Epoch 98/1000
74/74 [==============================] - 0s 260us/step - loss: 1964270.6149
Epoch 99/1000
74/74 [==============================] - 0s 163us/step - loss: 1964259.8446
Epoch 100/1000
74/74 [==

74/74 [==============================] - 0s 203us/step - loss: 1963417.0236
Epoch 180/1000
74/74 [==============================] - 0s 193us/step - loss: 1963406.5405
Epoch 181/1000
74/74 [==============================] - 0s 199us/step - loss: 1963395.9493
Epoch 182/1000
74/74 [==============================] - 0s 237us/step - loss: 1963385.4527
Epoch 183/1000
74/74 [==============================] - 0s 171us/step - loss: 1963375.1824
Epoch 184/1000
74/74 [==============================] - 0s 239us/step - loss: 1963364.6824
Epoch 185/1000
74/74 [==============================] - 0s 188us/step - loss: 1963354.2365
Epoch 186/1000
74/74 [==============================] - 0s 186us/step - loss: 1963343.8986
Epoch 187/1000
74/74 [==============================] - 0s 251us/step - loss: 1963333.2905
Epoch 188/1000
74/74 [==============================] - 0s 216us/step - loss: 1963322.8074
Epoch 189/1000
74/74 [==============================] - 0s 278us/step - loss: 1963312.3986
Epoch 190/1000

74/74 [==============================] - 0s 184us/step - loss: 1962480.2365
Epoch 270/1000
74/74 [==============================] - 0s 222us/step - loss: 1962469.8615
Epoch 271/1000
74/74 [==============================] - 0s 228us/step - loss: 1962459.4527
Epoch 272/1000
74/74 [==============================] - 0s 198us/step - loss: 1962449.0574
Epoch 273/1000
74/74 [==============================] - 0s 223us/step - loss: 1962438.8615
Epoch 274/1000
74/74 [==============================] - 0s 153us/step - loss: 1962428.3784
Epoch 275/1000
74/74 [==============================] - 0s 196us/step - loss: 1962418.0574
Epoch 276/1000
74/74 [==============================] - 0s 184us/step - loss: 1962407.6824
Epoch 277/1000
74/74 [==============================] - 0s 190us/step - loss: 1962397.2905
Epoch 278/1000
74/74 [==============================] - 0s 192us/step - loss: 1962386.9155
Epoch 279/1000
74/74 [==============================] - 0s 190us/step - loss: 1962376.5203
Epoch 280/1000

74/74 [==============================] - 0s 226us/step - loss: 1961549.4662
Epoch 360/1000
74/74 [==============================] - 0s 203us/step - loss: 1961539.1655
Epoch 361/1000
74/74 [==============================] - 0s 213us/step - loss: 1961528.8615
Epoch 362/1000
74/74 [==============================] - 0s 206us/step - loss: 1961518.5743
Epoch 363/1000
74/74 [==============================] - 0s 265us/step - loss: 1961508.1824
Epoch 364/1000
74/74 [==============================] - 0s 183us/step - loss: 1961497.8074
Epoch 365/1000
74/74 [==============================] - 0s 202us/step - loss: 1961487.5203
Epoch 366/1000
74/74 [==============================] - 0s 155us/step - loss: 1961477.2365
Epoch 367/1000
74/74 [==============================] - 0s 224us/step - loss: 1961466.9155
Epoch 368/1000
74/74 [==============================] - 0s 171us/step - loss: 1961456.5743
Epoch 369/1000
74/74 [==============================] - 0s 207us/step - loss: 1961446.2905
Epoch 370/1000

...

74/74 [==============================] - 0s 191us/step - loss: 1958786.0574
Epoch 629/1000
74/74 [==============================] - 0s 253us/step - loss: 1958775.6993
Epoch 630/1000
74/74 [==============================] - 0s 311us/step - loss: 1958765.5743
Epoch 631/1000
74/74 [==============================] - 0s 249us/step - loss: 1958755.2365
Epoch 632/1000
74/74 [==============================] - 0s 242us/step - loss: 1958745.0777
Epoch 633/1000
74/74 [==============================] - 0s 233us/step - loss: 1958734.7534
Epoch 634/1000
74/74 [==============================] - 0s 240us/step - loss: 1958724.5405
Epoch 635/1000
74/74 [==============================] - 0s 233us/step - loss: 1958714.3784
Epoch 636/1000
74/74 [==============================] - 0s 232us/step - loss: 1958704.1655
Epoch 637/1000
74/74 [==============================] - 0s 254us/step - loss: 1958693.7872
Epoch 638/1000
74/74 [==============================] - 0s 241us/step - loss: 1958683.6824
Epoch 639/1000

74/74 [==============================] - 0s 271us/step - loss: 1957865.5743
Epoch 719/1000
74/74 [==============================] - 0s 249us/step - loss: 1957855.3581
Epoch 720/1000
74/74 [==============================] - 0s 259us/step - loss: 1957845.2196
Epoch 721/1000
74/74 [==============================] - 0s 208us/step - loss: 1957834.8615
Epoch 722/1000
74/74 [==============================] - 0s 241us/step - loss: 1957824.6284
Epoch 723/1000
74/74 [==============================] - 0s 315us/step - loss: 1957814.3784
Epoch 724/1000
74/74 [==============================] - 0s 290us/step - loss: 1957804.3446
Epoch 725/1000
74/74 [==============================] - 0s 246us/step - loss: 1957794.1284
Epoch 726/1000
74/74 [==============================] - 0s 259us/step - loss: 1957784.0034
Epoch 727/1000
74/74 [==============================] - 0s 244us/step - loss: 1957773.6453
Epoch 728/1000
74/74 [==============================] - 0s 241us/step - loss: 1957763.5203
Epoch 729/1000

74/74 [==============================] - 0s 228us/step - loss: 1956946.8615
Epoch 809/1000
74/74 [==============================] - 0s 224us/step - loss: 1956936.6824
Epoch 810/1000
74/74 [==============================] - 0s 224us/step - loss: 1956926.4662
Epoch 811/1000
74/74 [==============================] - 0s 215us/step - loss: 1956916.2703
Epoch 812/1000
74/74 [==============================] - 0s 284us/step - loss: 1956906.1115
Epoch 813/1000
74/74 [==============================] - 0s 229us/step - loss: 1956895.8412
Epoch 814/1000
74/74 [==============================] - 0s 246us/step - loss: 1956885.5743
Epoch 815/1000
74/74 [==============================] - 0s 235us/step - loss: 1956875.5203
Epoch 816/1000
74/74 [==============================] - 0s 226us/step - loss: 1956865.2365
Epoch 817/1000
74/74 [==============================] - 0s 227us/step - loss: 1956855.1115
Epoch 818/1000
74/74 [==============================] - 0s 340us/step - loss: 1956844.8615
Epoch 819/1000

74/74 [==============================] - 0s 241us/step - loss: 1956029.5743
Epoch 899/1000
74/74 [==============================] - 0s 268us/step - loss: 1956019.3446
Epoch 900/1000
74/74 [==============================] - 0s 235us/step - loss: 1956009.1284
Epoch 901/1000
74/74 [==============================] - 0s 214us/step - loss: 1955999.0203
Epoch 902/1000
74/74 [==============================] - 0s 261us/step - loss: 1955988.8074
Epoch 903/1000
74/74 [==============================] - 0s 228us/step - loss: 1955978.7365
Epoch 904/1000
74/74 [==============================] - 0s 209us/step - loss: 1955968.4865
Epoch 905/1000
74/74 [==============================] - 0s 249us/step - loss: 1955958.2905
Epoch 906/1000
74/74 [==============================] - 0s 260us/step - loss: 1955948.1115
Epoch 907/1000
74/74 [==============================] - 0s 203us/step - loss: 1955937.9493
Epoch 908/1000
74/74 [==============================] - 0s 258us/step - loss: 1955927.7534
Epoch 909/1000

74/74 [==============================] - 0s 267us/step - loss: 1955113.6453
Epoch 989/1000
74/74 [==============================] - 0s 243us/step - loss: 1955103.4122
Epoch 990/1000
74/74 [==============================] - 0s 238us/step - loss: 1955093.3446
Epoch 991/1000
74/74 [==============================] - 0s 199us/step - loss: 1955083.1655
Epoch 992/1000
74/74 [==============================] - 0s 231us/step - loss: 1955073.0236
Epoch 993/1000
74/74 [==============================] - 0s 242us/step - loss: 1955062.8074
Epoch 994/1000
74/74 [==============================] - 0s 225us/step - loss: 1955052.7365
Epoch 995/1000
74/74 [==============================] - 0s 261us/step - loss: 1955042.5203
Epoch 996/1000
74/74 [==============================] - 0s 212us/step - loss: 1955032.3243
Epoch 997/1000
74/74 [==============================] - 0s 224us/step - loss: 1955022.1284
Epoch 998/1000
74/74 [==============================] - 0s 237us/step - loss: 1955011.8953
Epoch 999/1000

In [12]:
predictions = model.predict(xTrain)
"""
for i in predictions:
    ionContainer
    while """

'\nfor i in predictions:\n    ionContainer\n    while '

In [13]:
def printResults(dictionary, array):
    print("--------------------")
    j = 0
    for key, value in dictionary.items():
        for i in value:
            print(i, " | ", array[j])
            j += 1

for i, val in enumerate(predictions):
    dictionary = df.iloc[i]["ions"]
    printResults(dictionary, val)
        

--------------------
0  |  0.0
0  |  0.0
82.9000015258789  |  16.602093
70.9000015258789  |  16.176458
0  |  0.0
60.29999923706055  |  0.0
369.0  |  16.989677
22.200000762939453  |  0.0
86.0  |  16.436287
0  |  0.0
0  |  0.0
27.299999237060547  |  14.723925
0  |  0.0
0  |  0.0
28.399999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
41.20000076293945  |  0.0
254.10000610351562  |  16.827045
0  |  0.0
2000.300048828125  |  16.907291
1721.5  |  16.792114
4908.60009765625  |  17.07727
2042.0999755859375  |  0.0
3088.89990234375  |  17.124304
7947.7001953125  |  17.254755
376.20001220703125  |  0.0
0  |  0.0
0  |  0.0
82.9000015258789  |  0.0
70.9000015258789  |  16.179276
0  |  0.0
60.29999923706055  |  16.132818
369.0  |  16.760014
22.200000762939453  |  14.256109
86.0  |  0.0
0  |  0.0
0  |  0.0
27.299999237060547  |  14.77673
0  |  0.0
0  |  0.0
28.399999618530273  | 

27.899999618530273  |  14.839537
22.200000762939453  |  14.008792
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
57.5  |  16.086325
7.699999809265137  |  0.0
8.899999618530273  |  8.753829
59.400001525878906  |  15.938511
27.5  |  14.947816
212.5  |  16.842901
453.1000061035156  |  0.0
1622.199951171875  |  0.0
2818.39990234375  |  0.0
4202.7001953125  |  0.0
792.2999877929688  |  16.821987
2062.699951171875  |  0.0
3942.89990234375  |  16.891039
0  |  0.0
14.300000190734863  |  0.0
43.0  |  0.0
13.899999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
10.399999618530273  |  0.0
0  |  0.0
27.899999618530273  |  0.0
22.200000762939453  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
57.5  |  0.0
7.699999809265137  |  7.662704
8.899999618530273  |  8.784
59.400

7.699999809265137  |  0.0
8.899999618530273  |  8.753829
59.400001525878906  |  15.938511
27.5  |  14.947816
212.5  |  16.842901
453.1000061035156  |  0.0
1622.199951171875  |  0.0
2818.39990234375  |  0.0
4202.7001953125  |  0.0
792.2999877929688  |  16.821987
2062.699951171875  |  0.0
3942.89990234375  |  16.891039
0  |  0.0
14.300000190734863  |  0.0
43.0  |  0.0
13.899999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
10.399999618530273  |  0.0
0  |  0.0
27.899999618530273  |  0.0
22.200000762939453  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
57.5  |  0.0
7.699999809265137  |  7.662704
8.899999618530273  |  8.784
59.400001525878906  |  16.125147
27.5  |  0.0
212.5  |  0.0
453.1000061035156  |  17.095026
1622.199951171875  |  16.954683
2818.39990234375  |  0.0
4202.7001953125  |  16.901321
792.2999877929688  |  0.0
2062.699951171875  |  17.158176
3942.899902343

3088.89990234375  |  17.124304
7947.7001953125  |  17.254755
376.20001220703125  |  0.0
0  |  0.0
0  |  0.0
82.9000015258789  |  0.0
70.9000015258789  |  16.179276
0  |  0.0
60.29999923706055  |  16.132818
369.0  |  16.760014
22.200000762939453  |  14.256109
86.0  |  0.0
0  |  0.0
0  |  0.0
27.299999237060547  |  14.77673
0  |  0.0
0  |  0.0
28.399999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
41.20000076293945  |  15.771661
254.10000610351562  |  0.0
0  |  0.0
2000.300048828125  |  17.049664
1721.5  |  17.029293
4908.60009765625  |  0.0
2042.0999755859375  |  16.810522
3088.89990234375  |  17.251196
7947.7001953125  |  17.056253
376.20001220703125  |  16.971495
0  |  0.0
0  |  0.0
82.9000015258789  |  16.480007
70.9000015258789  |  16.561375
0  |  0.0
60.29999923706055  |  16.177681
369.0  |  16.8318
22.200000762939453  |  14.208302
86.0  |  16.335075
0  |  0.0
0

0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
41.20000076293945  |  15.771661
254.10000610351562  |  0.0
0  |  0.0
2000.300048828125  |  17.049664
1721.5  |  17.029293
4908.60009765625  |  0.0
2042.0999755859375  |  16.810522
3088.89990234375  |  17.251196
7947.7001953125  |  17.056253
376.20001220703125  |  16.971495
0  |  0.0
0  |  0.0
82.9000015258789  |  16.480007
70.9000015258789  |  16.561375
0  |  0.0
60.29999923706055  |  16.177681
369.0  |  16.8318
22.200000762939453  |  14.208302
86.0  |  16.335075
0  |  0.0
0  |  0.0
27.299999237060547  |  14.719937
0  |  0.0
0  |  0.0
28.399999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
41.20000076293945  |  15.69523
254.10000610351562  |  16.623278
0  |  0.0
2000.300048828125  |  0.0
1721.5  |  16.981512
4908.60009765625  |  16.907871
2042.09997

...

0  |  0.0
28.399999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
41.20000076293945  |  15.771661
254.10000610351562  |  0.0
0  |  0.0
2000.300048828125  |  17.049664
1721.5  |  17.029293
4908.60009765625  |  0.0
2042.0999755859375  |  16.810522
3088.89990234375  |  17.251196
7947.7001953125  |  17.056253
376.20001220703125  |  16.971495
0  |  0.0
0  |  0.0
82.9000015258789  |  16.480007
70.9000015258789  |  16.561375
0  |  0.0
60.29999923706055  |  16.177681
369.0  |  16.8318
22.200000762939453  |  14.208302
86.0  |  16.335075
0  |  0.0
0  |  0.0
27.299999237060547  |  14.719937
0  |  0.0
0  |  0.0
28.399999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
41.20000076293945  |  15.69523
254.10000610351562  |  16.623278
0  |  0.0
2000.300048828125  | 

0  |  0.0
0  |  0.0
28.399999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
41.20000076293945  |  15.69523
254.10000610351562  |  16.623278
0  |  0.0
2000.300048828125  |  0.0
1721.5  |  16.981512
4908.60009765625  |  16.907871
2042.0999755859375  |  16.846294
3088.89990234375  |  17.17952
7947.7001953125  |  16.83265
376.20001220703125  |  17.037212
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
57.5  |  0.0
7.699999809265137  |  7.660777
8.899999618530273  |  0.0
59.400001525878906  |  16.068056
27.5  |  0.0
212.5  |  16.840818
453.1000061035156  |  0.0
1622.199951171875  |  0.0
2818.39990234375  |  0.0
4202.7001953125  |  0.0
792.2999877929688  |  17.02903
2062.699951171875  |  0.0
3942.89990234375  |  0.0
0  |  0.0
14.300000190734863  |  12.128769
43.0  |  15.806582
13.899999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
10.399999618530273  |  9.954

27.5  |  0.0
212.5  |  16.840816
453.1000061035156  |  0.0
1622.199951171875  |  0.0
2818.39990234375  |  0.0
4202.7001953125  |  0.0
792.2999877929688  |  17.02903
2062.699951171875  |  0.0
3942.89990234375  |  0.0
0  |  0.0
14.300000190734863  |  12.128769
43.0  |  15.8065815
13.899999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
10.399999618530273  |  9.954509
0  |  0.0
27.899999618530273  |  14.839537
22.200000762939453  |  14.008791
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
57.5  |  16.086325
7.699999809265137  |  0.0
8.899999618530273  |  8.753829
59.400001525878906  |  15.938509
27.5  |  14.947816
212.5  |  16.842901
453.1000061035156  |  0.0
1622.199951171875  |  0.0
2818.39990234375  |  0.0
4202.7001953125  |  0.0
792.2999877929688  |  16.821987
2062.699951171875  |  0.0
3942.89990234375  |  16.891039
0  |  0.0
14.300000190734863  |  0.0
43.0  |  0.0
13.89999

59.400001525878906  |  15.938511
27.5  |  14.947816
212.5  |  16.842901
453.1000061035156  |  0.0
1622.199951171875  |  0.0
2818.39990234375  |  0.0
4202.7001953125  |  0.0
792.2999877929688  |  16.821987
2062.699951171875  |  0.0
3942.89990234375  |  16.891039
0  |  0.0
14.300000190734863  |  0.0
43.0  |  0.0
13.899999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
10.399999618530273  |  0.0
0  |  0.0
27.899999618530273  |  0.0
22.200000762939453  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
57.5  |  0.0
7.699999809265137  |  7.662704
8.899999618530273  |  8.784
59.400001525878906  |  16.125147
27.5  |  0.0
212.5  |  0.0
453.1000061035156  |  17.095026
1622.199951171875  |  16.954683
2818.39990234375  |  0.0
4202.7001953125  |  16.901321
792.2999877929688  |  0.0
2062.699951171875  |  17.158176
3942.89990234375  |  0.0
0  |  0.0
14.300000190734863  |  12.117044
43.

7.699999809265137  |  7.662704
8.899999618530273  |  8.784
59.400001525878906  |  16.125147
27.5  |  0.0
212.5  |  0.0
453.1000061035156  |  17.095026
1622.199951171875  |  16.954683
2818.39990234375  |  0.0
4202.7001953125  |  16.901321
792.2999877929688  |  0.0
2062.699951171875  |  17.158176
3942.89990234375  |  0.0
0  |  0.0
14.300000190734863  |  12.117044
43.0  |  15.59427
13.899999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
10.399999618530273  |  0.0
0  |  0.0
27.899999618530273  |  14.971294
22.200000762939453  |  14.074104
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
--------------------
0  |  0.0
0  |  0.0
82.9000015258789  |  16.602093
70.9000015258789  |  16.176458
0  |  0.0
60.29999923706055  |  0.0
369.0  |  16.989677
22.200000762939453  |  0.0
86.0  |  16.436287
0  |  0.0
0  |  0.0
27.299999237060547  |  14.723925
0  |  0.0
0  |  0.0
28.399999618530273  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  |  0.0
0  